In [1]:
import pandas as pd
import numpy as np
import torch
import os
import shutil
import sys
from PIL import Image
import utility_functions as uf

# testing area

In [2]:
#check if all the slides are processed to patches, and find the number of patches is less than 10000
directory_path = '/home/exon_storage1/aml_slide/patches/'
for file in os.listdir(directory_path):
    if  os.path.exists(directory_path+file+"/rightside_patch"):
        patches_path = directory_path+file+"/rightside_patch/"
        number_of_patches = len(os.listdir(patches_path))
        if number_of_patches <= 10000:
            print( file+" : "+str(len(os.listdir(patches_path))))
    else:
        print(file+" is not processed yet")

A474 : 9026
A1019 : 9448
A745 : 8617
A966 : 8314
A446 : 9581
A382 : 5811
A1093 : 5616
A168 : 7912
A991 : 5836
A978 : 7972
A1199 : 6236
A763 : 9567
A531 : 9305
A882 : 7313
A916 : 9651
A230 : 8442
A876 : 9850
A943 : 8468
A934 : 8564
A102 : 9910
A1122 : 3731
A820 : 6298
A1165 : 7669
A1027 : 2213
A1101 : 6445
A974 : 3849
A408 : 7267
A989 : 8523
A299 is not processed yet
A415 : 8853
A465 : 9344
A1047 : 9682
A993 : 9656
A506 : 8665


In [ ]:
#this is a script to remove the slides with the suffix of x or Rx, and rename the slides with the suffix of Lx
path = "/home/exon_storage1/aml_slide/patches/"
for file in os.listdir(path):
    detector = file.split("_")[-1]
    detector = detector.split(".")[0]
    if detector =="Lx":
        oldname = path+file
        newname = oldname.replace("_Lx","")
        print(oldname)
        os.rename(oldname, newname)
    elif detector == "x" or detector == "Rx":
        print(path+file)
        # os.remove(path+file)
        shutil.rmtree(path+file)

In [ ]:
#this script is used to normalize the patches and save them in a new folder, with a blurry filter
sys.path.insert(1, '/home/weber50432/AML_image_processing/HEnorm_python/')
from normalizeStaining import normalizeStaining , is_blurry
directory_path = '/home/weber50432/AML_image_processing/PyHIST/output/'
for file in os.listdir(directory_path):
    if  os.path.exists(directory_path+file+"/rightside_patch"):
        patches_path = directory_path+file+"/rightside_patch/"
        if os.path.exists(directory_path+file+"/rightside_patch_norm"):
            print(file + " rightside_patch_norm is already created !")
            continue
        else:
            os.mkdir(directory_path+file+"/rightside_patch_norm/")
        if os.path.exists(directory_path+file+"/rightside_patch_blurry"):
            print(file + " rightside_patch_blurry is already created !")
        else:
            os.mkdir(directory_path+file+"/rightside_patch_blurry/")
        patches_path_blurry = directory_path+file+"/rightside_patch_blurry/"
        patches_path_norm = directory_path+file+"/rightside_patch_norm/"
        for img in os.listdir(patches_path):
            if img.endswith(".png"):
                img_num = str(img)
                img_path = patches_path+img
                img = np.array(Image.open(img_path))
                Inorm, H, E = normalizeStaining(img = img,
                      saveFile = None,
                      Io = 240,
                      alpha = 1,
                      beta = 0.15)
                if is_blurry(Inorm):
                    Image.fromarray(Inorm).save(patches_path_blurry+img_num)
                    # os.remove(img_path)
                    print(img_num+" is blurry !")
                else:
                    Image.fromarray(Inorm).save(patches_path_norm+img_num)
                    print(img_num+" is saved !")